In [6]:
using AlphaVantage

client = AlphaVantage.GLOBAL[]
client.key = "9EJJOC93M35KBI63"


"9EJJOC93M35KBI63"

In [11]:
spy = time_series_daily_adjusted("SPY")
DataFrame(spy)

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount
,SubStrin…,Float64,Float64,Float64,Float64,Float64,Int64,Float64
1,2022-11-18,397.74,397.81,393.04,396.03,396.03,92922451,0.0
2,2022-11-17,390.46,394.95,390.14,394.24,394.24,74496255,0.0
3,2022-11-16,396.78,397.78,394.79,395.45,395.45,68508454,0.0
4,2022-11-15,401.15,402.31,394.49,398.49,398.49,93194452,0.0
5,2022-11-14,396.66,400.18,394.83,395.12,395.12,71893165,0.0
6,2022-11-11,395.59,399.35,393.61,398.51,398.51,93839872,0.0
7,2022-11-10,388.05,395.04,385.642,394.69,394.69,141455847,0.0
8,2022-11-09,379.93,381.14,373.61,374.13,374.13,78495470,0.0
9,2022-11-08,381.11,385.12,377.72,382.0,382.0,84641062,0.0


In [5]:
using DataFrames
tickers = ["B3SA3.SA", "PETR4.SA", "VALE3.SA", "ITUB4.SA", "BBAS3.SA", "USIM5.SA"]

returns_df = DataFrame()
for ticker in tickers
    df = DataFrame(time_series_daily_adjusted(ticker, outputsize="full", apikey=key))
    # limit the size of df to be the same as the others (to avoid errors)
    df = df[1:1000, :]
    returns = df.close[2:end] ./ df.close[1:end-1] .- 1
    returns_df[!, Symbol(ticker)] = returns
end

LoadError: MethodError: no method matching time_series_daily(::String; outputsize="full", apikey="9EJJOC93M35KBI63")
[0mClosest candidates are:
[0m  time_series_daily(::String; client, outputsize, datatype, parser) at C:\Users\Lucas\.julia\packages\AlphaVantage\c1lna\src\stock_time_series.jl:44[91m got unsupported keyword argument "apikey"[39m

In [ ]:
returns_df

In [ ]:
using CSV
CSV.write("returns.csv", returns_df)

In [ ]:
new_df = CSV.read("./returns.csv", DataFrame)


In [1]:
using Dualization, JuMP, HiGHS

σ = [100,200,300]
r_bar = [1,2,3]
Gamma = 5000



model = Model()
@variable(model, r[1:3] >= 0)
@variable(model, z[1:3] >= 0)
@constraint(model, negcon[i = 1:3], - r[i] + z[i]*σ[i] >= -r_bar[i])
@constraint(model, poscon[i = 1:3],  r[i] + z[i]*σ[i] >= r_bar[i])
@constraint(model, equalcon, sum(z) <= Gamma)
@constraint(model, lesscon[i = 1:3], z[i] ≤ 1)
@objective(model, Min, sum(r .* r_bar))

r[1] + 2 r[2] + 3 r[3]

In [2]:
dual_model = dualize(model, dual_names = DualNames("dual_", ""))

A JuMP Model
Maximization problem with:
Variables: 10
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: dual_equalcon_1, dual_lesscon[1]_1, dual_lesscon[2]_1, dual_lesscon[3]_1, dual_negcon[1]_1, dual_negcon[2]_1, dual_negcon[3]_1, dual_poscon[1]_1, dual_poscon[2]_1, dual_poscon[3]_1, r[1], r[2], r[3], z[1], z[2], z[3]

In [3]:
print(model)

In [4]:
print(dual_model)